# Behavior Clone Experiment: NVIDIA
We are using one lap of data, where driver tries to stay in the middle of the road. Now we are doing data processing and augmentation:
* normalizing the data and mean centering
* data augmentation by flipping the image and steering measurements
* images are cropped
* images are seen from multiple camera angles

The model is based on nvidia model. This model has the following 5 convolutional layers:
* convolutional layer 1: 24 5x5 filters with 2x2 stride
* convolutional layer 2: 36 5x5 filters with 2x2 stride
* convolutional layer 3: 48 5x5 filters with 2x2 stride
* convolutional layer 4: 64 3x3 filters with single stride
* convolutional layer 5: 64 3x3 filters with single stride with dropout
* fully connected layer


In [1]:
# load data from csv
import csv
import cv2
import numpy as np

lines = []
with open('../../../../data/bc_one_lap/driving_log.csv') as csvfile:
    rd = csv.reader(csvfile)
    for line in rd:
        lines.append(line)
        
        
images = []
measurements = []
# update path 
for line in lines:
    current_path = '../../../../data/bc_one_lap/IMG/' 
    img_center = cv2.imread(current_path + line[0].split('/')[-1])
    img_left = cv2.imread(current_path + line[1].split('/')[-1])
    img_right = cv2.imread(current_path + line[2].split('/')[-1])
    images.extend([img_center, img_left, img_right, cv2.flip(img_center, 1), cv2.flip(img_left, 1),
                   cv2.flip(img_right, 1)])
    steer_center = float(line[3])
    correction = 0.2 # parameter to tune
    steer_left = steer_center + correction
    steer_right = steer_center - correction
    measurements.extend([steer_center, steer_left, steer_right, steer_center*-1.0, steer_left*-1.0,
                         steer_right*-1.0])
    
# now convert to numpy arrays for keras
X_train = np.array(images)
y_train = np.array(measurements)

In [2]:
print(len(lines))

1486


In [2]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Convolution2D, Dropout, MaxPooling2D, Lambda, Cropping2D
input_shape=(160,320,3)

# most NN model with 2 layers and dropout
model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0))))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(250,activation='relu'))
model.add(Dense(1))
model.summary()

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 160, 320, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
cropping2d_1 (Cropping2D)        (None, 65, 320, 3)    0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 31, 158, 24)   1824        cropping2d_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 14, 77, 36)    21636       convolution2d_1[0][0]            
___________________________________________________________________________________________

In [10]:
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2,
          shuffle=True, nb_epoch=3)

Train on 7132 samples, validate on 1784 samples
Epoch 1/3
7132/7132 [==============================] - 61s - loss: 0.0260 - val_loss: 0.0135
Epoch 2/3
7132/7132 [==============================] - 61s - loss: 0.0186 - val_loss: 0.0165
Epoch 3/3
7132/7132 [==============================] - 66s - loss: 0.0155 - val_loss: 0.0236


In [11]:
model.save('../model.h5')

## Results
Using this model epochs were 3 times faster than the last experiment, also using this model the autonomous car went around track one for one lap. There were a few times when the car came close to the side of the road, but
the autonomous car corrrected itself. At times the autonomous car was eractic, so the next experiment is going to use more training data.

This can be seen in the video below.

In [13]:
%%HTML
<video width="320" height="240" controls>
  <source src="nvidia_onelap.mp4" type="video/mp4">
</video>